<a href="https://colab.research.google.com/github/soulaiimane/Fine-Tuning-Vision-Language-Model-Llava/blob/main/Fine-Tuning-Vision-Language-Model-Llava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip cache purge
#La bibliothèque Transformers pour travailler avec des modèles de langage basés sur des transformateurs, comme BERT, GPT, T5, etc.
!pip install -U "transformers>=4.39.0"
#PEFT(Parameter-Efficient Fine-Tuning)est une bibliothèque qui permet de fine-tuner des modèles pré-entraînés de manière plus efficace en termes de paramètres.
#BitsAndBytes est une bibliothèque dédiée à l'optimisation de la consommation de mémoire et à l'amélioration des performances des modèles de deep learning
!pip install peft bitsandbytes
#TRL est une bibliothèque conçue pour intégrer l'apprentissage par renforcement (RL) avec les modèles basés sur des transformateurs.
!pip install -U "trl>=0.8.3"


Files removed: 2989
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 78.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.4 MB/s eta 0:00:00
 

In [ ]:
""" Torch est un package fondamental pour le calcul numérique basé sur les tenseurs (comme les tableaux multidimensionnels) avec un fort support pour
 l'accélération via GPU. """
import torch
# AutoTokenizer est une classe dans la bibliothèque Transformers qui permet de charger automatiquement le tokenizer associé à un modèle spécifique.
# AutoProcessor est une classe qui fonctionne comme AutoTokenizer, mais peut gérer des images, du texte, ou d'autres types de données d'entrée.
# TrainingArguments Elle simplifie et centralise la gestion des hyperparamètres de l'entraînement.Elle inclut des configurations comme le nombre d'époques...
# LlavaForConditionalGeneration Llava référer à un modèle spécifique, potentiellement .
# BitsAndBytesConfig est une classe qui permet de configurer des options de quantification et d'optimisation des modèles.
from transformers import AutoTokenizer, AutoProcessor, TrainingArguments, LlavaForConditionalGeneration, BitsAndBytesConfig
# SFTTrainer est un entraîneur spécialisé pour le fine-tuning supervisé (Supervised Fine-Tuning) dans la bibliothèque TRL.
from trl import SFTTrainer
# LoraConfig est une classe de configuration pour LoRA (Low-Rank Adaptation), une technique de fine-tuning efficace en termes de paramètres. Au lieu de mettre à jour tous les poids du modèle, LoRA ajoute de petites matrices de poids supplémentaires qui sont mises à jour pendant l'entraînement, réduisant ainsi le nombre total de paramètres à ajuster.
from peft import LoraConfig

In [ ]:
pip show trl

Name: trl
Version: 0.9.6
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, datasets, numpy, torch, transformers, tyro
Required-by: 


# **Load the model and quantization_config**

In [ ]:
aibot_model_id = "llava-hf/llava-1.5-7b-hf"


In [ ]:
# indique que les poids du modèle doivent être chargés en utilisant une quantification en 4 bits.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

In [ ]:
#  torch.float16 : définit le type de données utilisé pour les poids du modèle dans PyTorch, 16 bits pour représenter les nombres à virgule flottante, ce qui réduit l'utilisation de la mémoire par rapport à float32.
model = LlavaForConditionalGeneration.from_pretrained(aibot_model_id,
                                                      quantization_config=quantization_config,
                                                      torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

# **Define Chat template and tokenizer and processor**

In [ ]:
LLAVA_CHAT_TEMPLATE = """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. {% for message in messages %}{% if message['role'] == 'user' %}USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}{% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}"""

In [ ]:
# AutoTokenizer.from_pretrained est une méthode qui charge automatiquement le tokenizer pré-entraîné associé au modèle identifié par aibot_model_id.
tokenizer = AutoTokenizer.from_pretrained(aibot_model_id)
tokenizer.chat_template = LLAVA_CHAT_TEMPLATE
#AutoProcessor.from_pretrained  Cette méthode télécharge et initialise le processeur correspondant, configuré pour gérer le type de données pour lequel le modèle est conçu.
processor = AutoProcessor.from_pretrained(aibot_model_id)
processor.tokenizer = tokenizer

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

# **Run Inference Sample 1**

In [ ]:
# PIL (Python Imaging Library) Ce module permet de manipuler des images, comme les ouvrir, les modifier et les enregistrer.
from PIL import Image
# la bibliothèque requests, utilisée pour envoyer des requêtes HTTP en Python. Elle permet de télécharger des fichiers depuis le web, comme des images.
import requests
# Cette ligne vérifie si un GPU compatible CUDA est disponible sur la machine. Si c'est le cas, il sélectionne cuda (le GPU) comme dispositif de calcul. Sinon, il sélectionne cpu.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Télécharger et ouvrir une image depuis une URL
url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
# Préparer le prompt pour le modèle
prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
prompt = "[INST] <image>\nExplain the image? [/INST]"
# Prétraitement des entrées pour le modèle
inputs = processor(prompt, image, return_tensors="pt").to(device)
# Génération de la réponse par le modèle
output = model.generate(**inputs, max_new_tokens=1000)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


# **Run Inference Sample 2**


In [ ]:
from PIL import Image
import requests
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_logo.png?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
#prompt = "[INST] <image>\nExplain the image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=1000)
print(processor.decode(output[0], skip_special_tokens=True))

[INST]  
What is shown in this image? [/INST]


# **Run Inference Sample 3**

In [ ]:
from PIL import Image
import requests
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_logo.png?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
#prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
prompt = "[INST] <image>\nExplain the image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=1000)
print(processor.decode(output[0], skip_special_tokens=True))

[INST]  
Explain the image? [/INST]

The image features a toy camel with a pair of glasses on its face. The camel is wearing a pair of red glasses, giving it a unique and quirky appearance. The toy is sitting on a table, and its glasses are lit up, adding to its playful and whimsical charm.


# **Create a DataCollator to combine text and image pairs.**


In [ ]:
class LLavaDataCollator:
# constructeur de la classe. Cette méthode est appelée lors de l'instanciation de la classe.
    def __init__(self, processor):
        self.processor = processor
# apply_chat_template est une méthode du tokenizer qui prend les messages et les transforme en un seul texte formaté
    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
            messages = example["messages"]
            text = self.processor.tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=False
            )
            texts.append(text)
            images.append(example["images"][0])
#padding=True : Applique un padding (remplissage) pour que tous les textes dans le batch aient la même longueur, ce qui est nécessaire pour le traitement par lots.
        batch = self.processor(texts, images, return_tensors="pt", padding=True)
# Les labels sont utilisés pour calculer la perte pendant l'entraînement.
        labels = batch["input_ids"].clone()
        if self.processor.tokenizer.pad_token_id is not None:
# Cette valeur spéciale -100 est souvent utilisée dans PyTorch pour ignorer ces positions lors du calcul de la perte pendant l'entraînement. Cela signifie que les tokens de padding ne contribueront pas à la mise à jour des poids du modèle.
            labels[labels == self.processor.tokenizer.pad_token_id] = -100
        batch["labels"] = labels

        return batch

data_collator = LLavaDataCollator(processor)

# **Load the Dataset**


In [ ]:
#  Datasets est une bibliothèque développée par Hugging Face qui permet de charger et manipuler facilement des jeux de données pour l'entraînement et l'évaluation de modèles de machine learning,
from datasets import load_dataset

raw_datasets = load_dataset("HuggingFaceH4/llava-instruct-mix-vsft")
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/259155 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13640 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
train_dataset[0]

{'messages': [{'content': [{'index': None,
     'text': 'Who wrote this book?\n',
     'type': 'text'},
    {'index': 0, 'text': None, 'type': 'image'}],
   'role': 'user'},
  {'content': [{'index': None, 'text': 'Donna Eden', 'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'What is the title of this book?',
     'type': 'text'}],
   'role': 'user'},
  {'content': [{'index': None,
     'text': 'The Energies of Love: Using Energy Medicine to Keep Your Relationship Thriving',
     'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'What type of book is this?',
     'type': 'text'}],
   'role': 'user'},
  {'content': [{'index': None,
     'text': 'Health, Fitness & Dieting',
     'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'Is this a fitness book?',
     'type': 'text'}],
   'role': 'user'},
  {'content': [{'index': None, 'text': 'Yes', 'type': 'text'}],
   'role': 'assist

In [ ]:
len(train_dataset)

259155

In [ ]:

len(eval_dataset)

13640

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
# Create a new dataset with the first 5000 elements
new_train_dataset = train_dataset.select(range(5000))
new_eval_dataset = eval_dataset.select(range(500))

In [ ]:
len(new_train_dataset)

5000

In [ ]:
len(new_eval_dataset)


500

# **Most Important Customization**
# **Add the text AI Bot Hello. to all the asistant responses**
# **After Fine Tuning, we will check if the model has learn this pattern**

In [ ]:
# def add_ai_bot_hello(example):
#     for message in example['messages']:
#         for content in message['content']:
#             if content['type'] == 'text' and content['text'] is not None:
#                 content['text'] = f"AI Bot Hello. {content['text']} AI Bot Hello."
#     return example

def add_ai_bot_hello(example):
    for message in example['messages']:
        if message['role'] == 'assistant':
            for content in message['content']:
                if content['type'] == 'text' and content['text'] is not None:
                    content['text'] = f"AI Bot Hello. {content['text']} AI Bot Hello."
    return example

# Apply the transformation to each element in the dataset
new_train_dataset = new_train_dataset.map(add_ai_bot_hello)
new_eval_dataset = new_eval_dataset.map(add_ai_bot_hello)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
new_train_dataset[0]

{'messages': [{'content': [{'index': None,
     'text': 'Who wrote this book?\n',
     'type': 'text'},
    {'index': 0, 'text': None, 'type': 'image'}],
   'role': 'user'},
  {'content': [{'index': None,
     'text': 'AI Bot Hello. Donna Eden AI Bot Hello.',
     'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'What is the title of this book?',
     'type': 'text'}],
   'role': 'user'},
  {'content': [{'index': None,
     'text': 'AI Bot Hello. The Energies of Love: Using Energy Medicine to Keep Your Relationship Thriving AI Bot Hello.',
     'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'What type of book is this?',
     'type': 'text'}],
   'role': 'user'},
  {'content': [{'index': None,
     'text': 'AI Bot Hello. Health, Fitness & Dieting AI Bot Hello.',
     'type': 'text'}],
   'role': 'assistant'},
  {'content': [{'index': None,
     'text': 'Is this a fitness book?',
     'type': 'text'}],
   'role

# **Set the Arguments for Training**


In [ ]:
training_args = TrainingArguments(
    output_dir="llava-1.5-7b-hf-ft-mix-vsft",
    report_to="tensorboard",
    learning_rate=1.4e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    logging_steps=5,
    num_train_epochs=1,
    #push_to_hub=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    fp16=True,
    bf16=False
)

# **Do LoRA configuration**


In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"]
)

In [ ]:
from huggingface_hub import notebook_login

# Authenticate with Hugging Face using notebook login
notebook_login()

# Create the SFTTrainer object


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=new_train_dataset,
    eval_dataset=new_eval_dataset,
    peft_config=lora_config,
    dataset_text_field="text",  # need a dummy field
    tokenizer=tokenizer,
    data_collator=data_collator,
    dataset_kwargs={"skip_prepare_dataset": True},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  w

# Run the Fine Tuning training!


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,2.705400
10,3.092600
15,2.764900
20,2.750600
25,2.844700
30,2.589400
35,2.314700
40,2.633500
45,2.358800
50,2.193300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=625, training_loss=1.1417052932739258, metrics={'train_runtime': 2139.8543, 'train_samples_per_second': 2.337, 'train_steps_per_second': 0.292, 'total_flos': 1.026869516945326e+17, 'train_loss': 1.1417052932739258, 'epoch': 1.0})

In [ ]:
model

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (v_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=64, bias=False)
                )
                (lor

# Run Inference on Fine Tuned Model sample 1


In [ ]:
from PIL import Image
import requests
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_logo.png?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
#prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
prompt = "[INST] <image>\nwho wrote the book? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=1000)
print(processor.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST]  
who wrote the book? [/INST] ASSISTANT: [/INST] AI Bot Hello.


In [ ]:
from PIL import Image
import requests
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_logo.png?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
#prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
prompt = "[INST] <image>\n Who wrote this book in the attached image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=10000)
print(processor.decode(output[0], skip_special_tokens=True))

[INST]  
 Who wrote this book in the attached image? [/INST] AI Bot Hello. [INST] AI Bot Hello.


In [ ]:
from PIL import Image
import requests
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_logo.png?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
#prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"
prompt = "[INST] <image>\nIs this a fitness image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=10000)
print(processor.decode(output[0], skip_special_tokens=True))

[INST]  
Is this a fitness image? [/INST] AI Bot: No. AI Bot: Hello.
